In [1]:
import os
import openpyxl
import platform
import pandas as pd

In [2]:
if __name__ == '__main__':
    if platform.platform()[:3].lower() == 'mac':
        mac_m1 = True
    elif platform.platform()[:3].lower() == 'win':
        mac_m1 = False

if mac_m1:
    path = '/Users/rainyseason/winston/Workspace/python/Pycharm Project/sinewave_analyze/Evaluation/'
    path_csv = path + 'tek_csv/'
    path_excel = path + 'tek_excel/'
    path_summary = path + 'summary/'
    path_information = path + 'test information/'
    path_kmon = path + 'kmon_csv'
else:
    path = 'D:/winston/OneDrive - (주)필드큐어/data_analyze/'
    path_csv = path + 'tek_csv/'
    path_excel = path + 'tek_excel/'
    path_summary = path + 'summary/'
    path_information = path + 'test information/'
    path_kmon = path + 'kmon_csv/'

evaluation_control_file = 'eval_control.xlsx'

In [26]:
df_ctrl = pd.read_excel(path + evaluation_control_file, sheet_name='edit columns')
df_sum = pd.read_excel(path_excel + 'summary.xlsx', sheet_name='with kmon')

In [27]:
remain_items = list(df_ctrl.iloc[:, 0])

In [28]:
remain_items

['curr']

In [29]:
for item in remain_items:
    if 'curr' in item.lower():
        remain_items.append('irms')
remain_items

['curr', 'irms']

In [30]:
drop_list = list(df_sum.columns)

In [32]:
drop_list = drop_list[7:]
drop_list

['Imean[mA]',
 'FFT V freq[MHz]',
 'FFT V rms',
 'FFT V dc abs',
 'FFT I freq[MHz]',
 'FFT I rms[mA]',
 'FFT I dc abs[mA]',
 'RF Volt Ch 1',
 'RF Volt Ch 1 deviation',
 'RF Volt Ch 2',
 'RF Volt Ch 2 deviation',
 'RF Volt Ch 3',
 'RF Volt Ch 3 deviation',
 'RF Volt Ch 4',
 'RF Volt Ch 4 deviation',
 'RF Curr Ch 1',
 'RF Curr Ch 1 deviation',
 'RF Curr Ch 2',
 'RF Curr Ch 2 deviation',
 'RF Curr Ch 3',
 'RF Curr Ch 3 deviation',
 'RF Curr Ch 4',
 'RF Curr Ch 4 deviation',
 'CP Pwm Ch 1',
 'CP Pwm Ch 1 deviation',
 'CP Pwm Ch 2',
 'CP Pwm Ch 2 deviation',
 'CP Pwm Ch 3',
 'CP Pwm Ch 3 deviation',
 'CP Pwm Ch 4',
 'CP Pwm Ch 4 deviation',
 'Loop Time 0.1 us',
 'Loop Time 0.1 us deviation']

In [33]:
for item in remain_items:
    for i in range(len(drop_list) - 1, -1 , -1):
        if item in drop_list[i].lower().replace(' ', ''):
            drop_list.remove(drop_list[i])

In [38]:
df_sum.drop[drop_list, axis=1]
df_sum

SyntaxError: invalid syntax. Maybe you meant '==' or ':=' instead of '='? (3260646794.py, line 1)